# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [3]:
pip install geoviews geopandas requests


Note: you may need to restart the kernel to use updated packages.


In [4]:
# Importing Dependencies and Setup
import hvplot.pandas
import pandas as pd
import geoviews as gv
import requests
from bokeh.io import show
from bokeh.plotting import figure

# Import API key
from api_keys import geoapify_key

In [5]:
# Loading the CSV file created in the Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Displaying sample data
city_data_df.head()

City_ID           City      Lat       Lng  Max Temp  Humidity  Cloudiness  \
0        0  blackmans bay -43.0167  147.3167     60.53        60           1   
1        1        stanley  54.8680   -1.6985     45.18        89           0   
2        2          tiksi  71.6872  128.8694     -0.51        99          86   
3        3     queenstown -31.8976   26.8753     54.81        92         100   
4        4     concepcion -36.8270  -73.0498     58.80        67          40   

   Wind Speed Country        Date  
0        1.01      AU  1731280921  
1       10.36      GB  1731280922  
2        3.00      RU  1731280923  
3        4.27      ZA  1731280924  
4       10.36      CL  1731280918

In [6]:
#Printing Columns to check their spelling and avoid any mispelling errors in code when calling the variables
print(city_data_df.columns)


Index(['City_ID', 'City', 'Lat', 'Lng', 'Max Temp', 'Humidity', 'Cloudiness',
       'Wind Speed', 'Country', 'Date'],
      dtype='object')


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [7]:
# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    size="Humidity",
    scale=1,
    color="City",
    tiles="OSM",
    frame_width=700,
    frame_height=500
)
 
# Displaying the map
map_plot


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [8]:
# Narrowing down cities that fit criteria and drop any results with null values
filtered_cities = city_data_df[(city_data_df['Humidity'] > 50) & (city_data_df['Max Temp'] < 30)]

# Droping any rows with null values
filtered_cities_cleaned = filtered_cities.dropna()

# Display sample data
filtered_cities_cleaned.head()  # You can also use .sample() to display a random sample


City_ID          City      Lat       Lng  Max Temp  Humidity  Cloudiness  \
2         2         tiksi  71.6872  128.8694     -0.51        99          86   
6         6  longyearbyen  78.2186   15.6401     26.44        93          75   
7         7       aasiaat  68.7098  -52.8699     13.26        99          90   
11       11       udachny  66.4167  112.4000      3.33        95          94   
12       12     vorgashor  67.5833   63.9500      3.20        98          84   

    Wind Speed Country        Date  
2         3.00      RU  1731280923  
6         3.44      SJ  1731280928  
7         4.88      GL  1731280929  
11        4.90      RU  1731280744  
12        8.23      RU  1731280935

### Step 3: Create a new DataFrame called `hotel_df`.

In [9]:
# 
hotel_df = city_data_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# 
hotel_df['Hotel Name'] = None  # Initialize with None (or np.nan if you prefer)

# 
hotel_df.head()  # Use .sample() if you prefer a random sample


City Country      Lat       Lng  Humidity Hotel Name
0  blackmans bay      AU -43.0167  147.3167        60       None
1        stanley      GB  54.8680   -1.6985        89       None
2          tiksi      RU  71.6872  128.8694        99       None
3     queenstown      ZA -31.8976   26.8753        92       None
4     concepcion      CL -36.8270  -73.0498        67       None

### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [10]:
# Setting parameters to search for a hotel
radius = 50000  # Setting a search radius of 50 km

# Creating a params dictionary to hold the Geoapify API key and other filters
params = {
    "categories": "accommodation.hotel",  # Searching for hotels
    "apiKey": geoapify_key,    
    "limit": 1                            # Limitting results to the first 5 hotels found
}

# Printing a message to follow up the hotel search
print("Starting hotel search")

# Iterating through the hotel_df DataFrame
for index, row in hotel_df.iterrows():

    # Getting the city's name
    city = row["City"]
    
    # Getting latitude and longitude from the DataFrame for each city
    latitude = row['Lat']
    longitude = row['Lng']

    # Adding the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"  # Setting filter as circle with coordinates
    params.pop("bias", None)  # Remove bias for now, just to test

    # Setting base URL for the Geoapify API
    base_url = "https://api.geoapify.com/v2/places"

    # Making an API request using the params dictionary
    response = requests.get(base_url, params=params)

    # Converting the API response to JSON format
    name_address = response.json()

    # Debugging: Print API response status and content
    print(f"Response for {city}: Status {response.status_code}")
    print(name_address)

    # Grabbing the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        features = name_address.get("features", [])
        if features:
            hotel_df.loc[index, "Hotel Name"] = features[0]["properties"]["name"]
        else:
            hotel_df.loc[index, "Hotel Name"] = "No hotel found"
    except Exception as e:
        print(f"Error in city {city}: {e}")
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Logging the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Displaying sample data
hotel_df.head()


Starting hotel search
Response for blackmans bay: Status 200
{'type': 'FeatureCollection', 'features': [{'type': 'Feature', 'properties': {'name': 'Hotel Grand Chancellor Hobart', 'country': 'Australia', 'country_code': 'au', 'state': 'Tasmania', 'city': 'Hobart', 'municipality': 'City of Hobart', 'postcode': '7000', 'suburb': 'Hobart', 'street': 'Davey Street', 'housenumber': '1', 'lon': 147.33337188844695, 'lat': -42.880809600000006, 'state_code': 'TAS', 'formatted': 'Hotel Grand Chancellor Hobart, 1 Davey Street, Hobart TAS 7000, Australia', 'address_line1': 'Hotel Grand Chancellor Hobart', 'address_line2': '1 Davey Street, Hobart TAS 7000, Australia', 'categories': ['accommodation', 'accommodation.hotel', 'building', 'building.accommodation'], 'details': ['details', 'details.building', 'details.contact', 'details.wiki_and_media'], 'datasource': {'sourcename': 'openstreetmap', 'attribution': '© OpenStreetMap contributors', 'license': 'Open Database License', 'url': 'https://www.open

City Country      Lat       Lng  Humidity  \
0  blackmans bay      AU -43.0167  147.3167        60   
1        stanley      GB  54.8680   -1.6985        89   
2          tiksi      RU  71.6872  128.8694        99   
3     queenstown      ZA -31.8976   26.8753        92   
4     concepcion      CL -36.8270  -73.0498        67   

                      Hotel Name  
0  Hotel Grand Chancellor Hobart  
1     Hilton Newcastle Gateshead  
2                        Арктика  
3                   Queens Hotel  
4      Wyndham Concepción Pettra

In [11]:
#Rechecking the data in hotel df
hotel_df.head()

City Country      Lat       Lng  Humidity  \
0  blackmans bay      AU -43.0167  147.3167        60   
1        stanley      GB  54.8680   -1.6985        89   
2          tiksi      RU  71.6872  128.8694        99   
3     queenstown      ZA -31.8976   26.8753        92   
4     concepcion      CL -36.8270  -73.0498        67   

                      Hotel Name  
0  Hotel Grand Chancellor Hobart  
1     Hilton Newcastle Gateshead  
2                        Арктика  
3                   Queens Hotel  
4      Wyndham Concepción Pettra

### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [12]:
# Configure the map plot
map_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    size="Humidity",
    scale=1,
    color="City",
    tiles="OSM",
    frame_width=700,
    frame_height=500,
    hover_cols=["Hotel Name", "Country"],  # Including columns for the hover
   # tools=[hover_tool]  # Use the custom hover tool
)
 
# Displaying the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)